In [63]:
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

_ : bool = load_dotenv(find_dotenv())
client : OpenAI = OpenAI()

In [64]:
# custom function
import json
def getCurrentWeather(location:str, unit:str='fahrenheit')-> str | dict | None :
    """Get current weather in the given location"""
    if 'tokyo' in location.lower():
        return json.dumps({'location' : 'Tokyo', 'unit' : 'fahrenheit', 'temprature': '72'}),
    elif 'los angeles' in location.lower():
        return json.dumps({'location' : 'Los Angeles', 'unit' : 'celsius', 'temprature': '22'}),
    elif 'paris' in location.lower():
        return json.dumps({'location' : 'Paris', 'unit' : 'celsius', 'temprature': '12'}),
    else : 
        return json.dumps({'location' : location, 'unit' : 'unknown'})
    

def getNickName(location: str)-> str:
        '''Get nickname of City'''
        if 'tokyo' in location.lower():
            return "tk",
        elif 'los angeles' in location.lower():
            return "ls",
        elif 'paris' in location.lower():
            return "pr",
        else:
            return location

In [65]:
import json

def show_json(message, obj):
    display(message, json.loads(obj.model_dump_json()))

In [66]:
from openai.types.beta import Assistant

assistant: Assistant = client.beta.assistants.create(
  instructions="You are a weather bot. Use the provided functions to answer questions.",
  model="gpt-3.5-turbo-1106",
  tools=[{
      "type": "function",
    "function": {
      "name": "getCurrentWeather",
      "description": "Get the weather in location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
          "unit": {"type": "string", "enum": ["c", "f"]}
        },
        "required": ["location"]
      }
    }
  }, {
    "type": "function",
    "function": {
      "name": "getNickname",
      "description": "Get the nickname of a city",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
        },
        "required": ["location"]
      }
    } 
  },      
      
  

  ]
),

In [67]:
#thread

from openai.types.beta.thread import Thread

thread : Thread = client.beta.threads.create()

In [68]:
#thread message
from openai.types.beta.threads.thread_message import ThreadMessage

message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role='user',
    content= 'How is the weather in Los Angles?',
)

In [70]:
# 5. run
from openai.types.beta.threads.run import Run

run : Run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    
)

AttributeError: 'tuple' object has no attribute 'id'

In [ ]:
available_functions = {
    'getCurrentWeather' : getCurrentWeather,
    'getNickName' : getNickName,
}

In [27]:
import time

  
while True:
    
    runStatus = client.beta.threads.runs.retrieve(thread_id=thread.id,
                                                  run_id=run.id)
  
    run_steps = client.beta.threads.runs.steps.list(thread_id=thread.id, run_id=run.id)
  
    print(runStatus.status ,'.....')

    # This means run is making a function call   
    if runStatus.status == "requires_action":
        print(runStatus.status ,'.....')
        print("Status: ", "requires_action")
        show_json("submit_tool_outputs", runStatus.required_action)
        if runStatus.required_action.submit_tool_outputs and runStatus.required_action.submit_tool_outputs.tool_calls:
            print("toolCalls present:")
            toolCalls = runStatus.required_action.submit_tool_outputs.tool_calls

            tool_outputs = []
            for toolcall in toolCalls:
                function_name = toolcall.function.name
                function_args = json.loads(toolcall.function.arguments)
                
                if function_name in available_functions:
                    
                    
                    function_to_call = available_functions[function_name]
                    print(function_to_call,function_to_call.__name__=="getCurrentWeather","================================================================")
                  
                    if function_to_call.__name__ == "getCurrentWeather":
                        
                        response = function_to_call(
                        location=function_args.get("location"),
                        unit=function_args.get("unit")
                        )
                        
                        
                        tool_outputs.append({
                                  "tool_call_id": toolcall.id,
                                  "output": response
                              })
                    
                    elif function_to_call.__name__ == "getNickname":
                        response = function_to_call(
                          location=function_args.get("location")
                          )
                        tool_outputs.append({
                          "tool_call_id": toolcall.id,
                          "output": response,
                              })
            print(tool_outputs,">>>>>") 
        
            client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=tool_outputs)
      
    elif runStatus.status == "completed":
        
        print("completed...........logic")
        messages: list[ThreadMessage] = client.beta.threads.messages.list(thread_id=thread.id)
        for message in messages.data:
            role_label = "User" if message.role == "user" else "Assistant"
            message_content = message.content[0].text.value
            print(f"{role_label}: {message_content}\n")
        break  

    elif run.status == "failed":
      print("Run failed.")
      break

    elif run.status in ["in_progress", "queued"]:
      print(f"Run is {run.status}. Waiting...")
      time.sleep(5)  # Wait for 5 seconds before checking again

    else:
      print(f"Unexpected status: {run.status}")
      break

requires_action .....
requires_action .....
Status:  requires_action
toolCalls present:
<function getCurrentWeather at 0x0000020894C9E480> True ================================================================
<function getNichName at 0x0000020894689DA0> False ================================================================
[{'tool_call_id': 'call_fc0E5dNFUZ3qIBAlMphJovtg', 'output': ('{"location": "Karachi", "unit": "celsius", "temprature": "32"}',)}] >>>>>


BadRequestError: Error code: 400 - {'error': {'message': '1 validation error for Request\nbody -> tool_outputs -> 0 -> output\n  str type expected (type=type_error.str)', 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [50]:
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

_ : bool = load_dotenv(find_dotenv())

client : OpenAI = OpenAI()


In [52]:
# 1. create a cusotm function

def getCurrentWeather(location : str, unit : str = 'fahrenheit') -> str | dict | None :
    """ Get current weather in a given location"""
    if 'lahore' in location.lower():
        return json.dumps({'location': 'Lahore', "unit": 'fahrenheit', 'temprature' : '72'}),
    elif 'faisalabad' in location.lower():
        return json.dumps({'location': 'Faisalabad', "unit": 'celsius', 'temprature' : '10'}),
    elif 'karachi' in location.lower():
        return json.dumps({'location': 'Karachi', "unit": 'celsius', 'temprature' : '32'}),
    else :
        return json.dumps({'location':location, "temprature": 'Unknown'})


def getNichName(location:str)-> str :
    """Get Nick name of city"""
    if 'lahore' in location.lower():
        return 'LA',    
    elif 'faisalabad' in location.lower():
        return 'FA', 
    elif 'karachi' in location.lower():
        return 'KA',
    else:
        return location


In [53]:
import json
def show_json(message,obj) :
    return (message, json.loads(obj.model_dump_json()))

In [54]:
# 2.create assistant 

from openai.types.beta import Assistant

assistant : Assistant = client.beta.assistants.create(
    instructions= ' You are a weather assistant chatbot.please tell us weather of given location , from given knowladge ',
    model='gpt-3.5-turbo-1106',
    tools= [
        {
            'type': 'function',
            'function': {
                'name' : 'getCurrentWeather',
                'description' : 'Get current weather in a given location',
                'parameters': {
                    'type' : 'object',
                    'properties': {
                        'location' : {'type': 'string', 'description': 'The name of city or state e.g. Lahore, LA '},
                        'unit' : {'type': 'string', 'enum':['f','c']}
                    },
                    'requires':['location']
                },

            }
        },{
            'type' : 'function',
            'function':{
                'name': 'getNichName',
                'description':'Get the Nick name of city',
                'parameters':{
                    'type': 'object',
                    'properties': {
                        'location':{'type': 'string', 'description':'The name of city or state e.g. Lahore , LA'},
                    },
                    'required': ['loation'],
                },
            }
        }
    ],
       
    )

In [55]:
# 3.create a thread

from openai.types.beta.thread import Thread

thread: Thread = client.beta.threads.create()

In [56]:
 #4. Thread message

from openai.types.beta.threads.thread_message import ThreadMessage
message : ThreadMessage = client.beta.threads.messages.create(
    thread_id=thread.id,
    role='user',
    content='What,s the weather like in Karachi?'   
)

In [57]:
# 5. run
from openai.types.beta.threads.run import Run

run : Run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions='please address the user as a aamir. he has a premium account',
)

In [24]:
available_functions={
    'getCurrentWeather' : getCurrentWeather,
    'getNichName' : getNichName,
}

In [26]:
import time

  # Loop until the run completes or requires action
while True:
    runStatus = client.beta.threads.runs.retrieve(thread_id=thread.id,
                                                  run_id=run.id)
    # Add run steps retrieval here for debuging
    run_steps = client.beta.threads.runs.steps.list(thread_id=thread.id, run_id=run.id)
    # show_json("Run Steps:", run_steps)
    print(runStatus.status ,'.....')

    # This means run is making a function call   
    if runStatus.status == "requires_action":
        print(runStatus.status ,'.....')
        print("Status: ", "requires_action")
        show_json("submit_tool_outputs", runStatus.required_action)
        if runStatus.required_action.submit_tool_outputs and runStatus.required_action.submit_tool_outputs.tool_calls:
            print("toolCalls present:")
            toolCalls = runStatus.required_action.submit_tool_outputs.tool_calls

            tool_outputs = []
            for toolcall in toolCalls:
                function_name = toolcall.function.name
                function_args = json.loads(toolcall.function.arguments)
                
                if function_name in available_functions:
                    
                    
                    function_to_call = available_functions[function_name]
                    print(function_to_call,function_to_call.__name__=="getCurrentWeather","================================================================")
                  
                    if function_to_call.__name__ == "getCurrentWeather":
                        
                        response = function_to_call(
                        location=function_args.get("location"),
                        unit=function_args.get("unit")
                        )
                        
                        
                        tool_outputs.append({
                                  "tool_call_id": toolcall.id,
                                  "output": response
                              })
                    
                    elif function_to_call.__name__ == "getNickname":
                        response = function_to_call(
                          location=function_args.get("location")
                          )
                        tool_outputs.append({
                          "tool_call_id": toolcall.id,
                          "output": response,
                              })
            print(tool_outputs,">>>>>") 
            # Submit tool outputs and update the run
            client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=tool_outputs)
      
    elif runStatus.status == "completed":
        # List the messages to get the response
        print("completed...........logic")
        messages: list[ThreadMessage] = client.beta.threads.messages.list(thread_id=thread.id)
        for message in messages.data:
            role_label = "User" if message.role == "user" else "Assistant"
            message_content = message.content[0].text.value
            print(f"{role_label}: {message_content}\n")
        break  # Exit the loop after processing the completed run

    elif run.status == "failed":
      print("Run failed.")
      break

    elif run.status in ["in_progress", "queued"]:
      print(f"Run is {run.status}. Waiting...")
      time.sleep(5)  # Wait for 5 seconds before checking again

    else:
      print(f"Unexpected status: {run.status}")
      break


requires_action .....
requires_action .....
Status:  requires_action
toolCalls present:
<function getCurrentWeather at 0x0000020894C9E480> True ================================================================
<function getNichName at 0x0000020894689DA0> False ================================================================
[{'tool_call_id': 'call_fc0E5dNFUZ3qIBAlMphJovtg', 'output': ('{"location": "Karachi", "unit": "celsius", "temprature": "32"}',)}] >>>>>


BadRequestError: Error code: 400 - {'error': {'message': '1 validation error for Request\nbody -> tool_outputs -> 0 -> output\n  str type expected (type=type_error.str)', 'type': 'invalid_request_error', 'param': None, 'code': None}}